# Einstein's equations of general relativity in the [BSSN](http://www2.yukawa.kyoto-u.ac.jp/~yuichiro.sekiguchi/3+1.pdf) formalism, in ***curvilinear*** coordinates, using a covariant reference metric approach: C code generation of the evolution equations' right-hand sides

### ***Citations***: Generic curvilinear coordinate reference metric approach matches that of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658), which is an extension of the spherical coordinate reference metric approach of [Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632), which builds upon the covariant "Lagrangian" BSSN formalism of [Brown (2009)](https://arxiv.org/abs/0902.3652). *See also citations within each article.*

**Background**: NRPy+'s original purpose was to be an easy-to-use code capable of generating Einstein's equations in a broad class of [singular](https://en.wikipedia.org/wiki/Coordinate_singularity), curvilinear coordinate systems, where the user need only input the scale factors of the underlying reference metric. Upon generating these equations, NRPy+ would then leverage SymPy's [common-expression-elimination (CSE)](https://en.wikipedia.org/wiki/Common_subexpression_elimination) and C code generation routines, coupled to its own [single-instruction, multiple-data (SIMD)](https://en.wikipedia.org/wiki/SIMD) functions, to generate highly-optimized C code.

This tutorial module demonstrates how Einstein's equations of general relativity in this formulation are constructed and output within NRPy+. As Einstein's equations in this formalism take the form of highly nonlinear, coupled *wave equations*, the [tutorial module on the scalar wave equation in curvilinear coordinates](Tutorial-ScalarWaveCurvilinear.ipynb) is *required* reading before beginning this module. That module, as well as its own prerequisite [module on reference metrics within NRPy+](Tutorial-Reference_Metric.ipynb) provides the needed overview of how NRPy+ handles reference metrics.

In summary, the equations we wish to input into NRPy+ are as follows (Eqs. 11 and 12 in [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)):

\begin{align}
  \partial_{\perp} \bar{\gamma}_{i j} {} = {} & \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right ) + 2 \hat{D}_{(i} \beta_{j)} - 2 \alpha \bar{A}_{i j} \; , \\
  \partial_{\perp} \bar{A}_{i j} {} = {} & -\frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K \nonumber \\
  & + e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi \right . \nonumber \\
    & \left . + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}} \; , \\
  \partial_{\perp} \phi {} = {} & \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) \; , \\
  \partial_{\perp} K {} = {} & \frac{1}{3} \alpha K^{2} + \alpha \bar{A}_{i j} \bar{A}^{i j} \nonumber \\
  & - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right ) \; , \\
  \partial_{\perp} \bar{\Lambda}^{i} {} = {} & \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \bar{A}^{j k} \Delta_{j k}^{i} \nonumber \\
  & -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K \\
\partial_{0} \alpha &= -2 \alpha K \\
  \partial_{0} \beta^{i} &= B^{i} \\
  \partial_{0} B^{i} &= \frac{3}{4} \partial_{0} \bar{\Lambda}^{i} - \eta B^{i} \; .
\end{align}
where 
* the $\text{TF}$ superscript denotes the trace-free part.
* $\bar{\gamma}_{ij} = \varepsilon_{i j} + \hat{\gamma}_{ij}$, where $\bar{\gamma}_{ij} = e^{4\phi} \gamma_{ij}$ is the conformal metric, $\gamma_{ij}$ is the physical metric (see below), and $\varepsilon_{i j}$ encodes information about the non-hatted metric.
* $\gamma_{ij}$, $\beta^i$, and $\alpha$ are the physical (as opposed to conformal) spatial 3-metric, shift vector, and lapse, respectively, which may be defined via the 3+1 decomposition line element (in [$G=c=1$ units](https://en.wikipedia.org/wiki/Planck_units)):
$$ds^2 = -\alpha^2 dt^2 + \gamma_{ij}\left(dx^i + \beta^i dt\right)\left(dx^j + \beta^j dt\right).$$
* $\bar{R}_{ij}$ is the conformal Ricci tensor, computed via
\begin{align}
  \bar{R}_{i j} {} = {} & - \frac{1}{2} \bar{\gamma}^{k l} \hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j} + \bar{\gamma}_{k(i} \hat{D}_{j)} \bar{\Lambda}^{k} + \Delta^{k} \Delta_{(i j) k} \nonumber \\
  & + \bar{\gamma}^{k l} \left (2 \Delta_{k(i}^{m} \Delta_{j) m l} + \Delta_{i k}^{m} \Delta_{m j l} \right ) \; .
\end{align}
* $\partial_{\perp} = \partial_t - \mathcal{L}_\beta$; $\mathcal{L}_\beta$ is the [Lie derivative](https://en.wikipedia.org/wiki/Lie_derivative) along the shift vector $\beta^i$.
* $\partial_0 = \partial_t - \beta^i \partial_i$ is an advective time derivative.
* $\hat{D}_j$ is the [covariant derivative](https://en.wikipedia.org/wiki/Covariant_derivative) with respect to the reference metric $\hat{\gamma}_{ij}$.
* $\bar{D}_j$ is the [covariant derivative](https://en.wikipedia.org/wiki/Covariant_derivative) with respect to the barred spatial 3-metric $\bar{\gamma}_{ij}$
* $\Delta^i_{jk}$ is the tensor constructed from the difference of barred and hatted Christoffel symbols:
$$\Delta^i_{jk} = \bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk}$$
    * The related quantity $\Delta^i$ is defined $\Delta^i \equiv \bar{\gamma}^{jk} \Delta^i_{jk}$.
* $\bar{A}_{ij}$ is the conformal, trace-free extrinsic curvature: 
$$\bar{A}_{ij} = e^{-4\phi} \left(K_{ij} - \frac{1}{3}\gamma_{ij} K\right),$$
where $K$ is the trace of the extrinsic curvature $K_{ij}$.

### The Lie derivatives

In this section, we provide explicit expressions for the [Lie derivatives](https://en.wikipedia.org/wiki/Lie_derivative) $\mathcal{L}_\beta$ appearing inside the $\partial_\perp = \partial_t - \mathcal{L}_\beta$ operators for $\left\{\varepsilon_{i j},\bar{A}_{i j},W, K, \bar{\Lambda}^{i}\right\}$.

In short, the Lie derivative of tensor weight $w$ is given by (from [the wikipedia article on Lie derivatives](https://en.wikipedia.org/wiki/Lie_derivative))
\begin{align}
(\mathcal {L}_X T) ^{a_1 \ldots a_r}{}_{b_1 \ldots b_s} &= X^c(\partial_c T^{a_1 \ldots a_r}{}_{b_1 \ldots b_s}) \\
&\quad - (\partial_c X ^{a_1}) T ^{c a_2 \ldots a_r}{}_{b_1 \ldots b_s} - \ldots - (\partial_c X^{a_r}) T ^{a_1 \ldots a_{r-1}c}{}_{b_1 \ldots b_s} \\
 &\quad  +  (\partial_{b_1} X^c) T ^{a_1 \ldots a_r}{}_{c b_2 \ldots b_s} + \ldots + (\partial_{b_s} X^c) T ^{a_1 \ldots a_r}{}_{b_1 \ldots b_{s-1} c} + w (\partial_{c} X^c) T ^{a_1 \ldots a_r}{}_{b_1 \ldots b_{s}}
\end{align}

Thus to evaluate the Lie derivative, one must first know the tensor density weight $w$ for each tensor. In this formulation of Einstein's equations, **all evolved quantities have density weight $w=0$**, so according to the definition of Lie derivative above,
\begin{align}
\mathcal{L}_\beta \bar{\gamma}_{ij} &= \beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik}, \\
\mathcal{L}_\beta \bar{A}_{ij} &= \beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik}, \\
\mathcal{L}_\beta K &= \beta^k \partial_k K, \\
\mathcal{L}_\beta \phi &= \beta^k \partial_k \phi, \\
\mathcal{L}_\beta \bar{\Lambda}^i &= \beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k
\end{align}


## Numerical Implementation: Rescaling tensors to factor out coordinate singularities

While the above evolution equations are properly covariant (with the exception of the shift condition, which is a [freely specifiable gauge quantity](https://en.wikipedia.org/wiki/Gauge_fixing)), components of the rank-1 and rank-2 tensors $\varepsilon_{i j}$, $\bar{A}_{i j}$, and $\bar{\Lambda}^{i}$ will drop to zero (destroying information) or diverge (to $\infty$) at coordinate singularities. However, this behavior is well-understood in terms of the scale factors of the reference metric, enabling us to define rescaled version of these quantities that are well behaved (so that, e.g., they can be finite differenced).

For example, given a smooth vector *in a 3D Cartesian basis* $\bar{\Lambda}^{i}$, all components $\bar{\Lambda}^{x}$, $\bar{\Lambda}^{y}$, and $\bar{\Lambda}^{z}$ will be smooth (by assumption). When changing the basis to spherical coordinates (applying the appropriate Jacobian matrix transformation), we will find that since $\phi = \arctan(y/x)$, $\bar{\Lambda}^{\phi}$ is given by

\begin{align}
\bar{\Lambda}^{\phi} &= \frac{\partial \phi}{\partial x} \bar{\Lambda}^{x} + 
\frac{\partial \phi}{\partial y} \bar{\Lambda}^{y} + 
\frac{\partial \phi}{\partial z} \bar{\Lambda}^{z} \\
&= -\frac{y}{\sqrt{x^2+y^2}} \bar{\Lambda}^{x} + 
\frac{x}{\sqrt{x^2+y^2}} \bar{\Lambda}^{y} \\
&= -\frac{y}{r \sin\theta} \bar{\Lambda}^{x} + 
\frac{x}{r \sin\theta} \bar{\Lambda}^{y}.
\end{align}

Thus $\bar{\Lambda}^{\phi}$ diverges at all points where $r\sin\theta=0$ due to the $\frac{1}{r\sin\theta}$ that appear in the Jacobian transformation. 

This divergence might pose no problem on cell-centered grids that avoid $r \sin\theta=0$, except that the BSSN equations require that *first and second derivatives* of these quantities be taken. Usual strategies for numerical approximation of these derivatives (e.g., finite difference methods) will not converge with increased numerical sampling of the functions at points near where the functions diverge.

However, notice that if we define $\lambda^{\phi}$ such that

$$\bar{\Lambda}^{\phi} = \frac{1}{r\sin\theta} \lambda^{\phi},$$

then $\lambda^{\phi}$ will be smooth as well. 

Avoiding such singularities can be generalized, so long as $\lambda^{\phi}$ is defined as:

$$\bar{\Lambda}^{i} = \frac{\lambda^i}{\text{scalefactor[i]}} ,$$

where scalefactor\[i\] is the $i$th scale factor in the given coordinate system. In an identical fashion, we define the smooth versions of $\beta^i$ and $B^i$ to be $\mathcal{V}^i$ and $\mathcal{B}^i$, respectively. We refer to $\mathcal{V}^i$ and $\mathcal{B}^i$ as vet\[i\] and bet\[i\] respectively in the code after the Hebrew letters that bear some resemblance. 

Similarly, we define the smooth versions of $\bar{A}_{ij}$ and $\bar{\varepsilon}_{ij}$ ($a_{ij}$ and $h_{ij}$, respectively) via

\begin{align}
\bar{A}_{ij} &= \text{scalefactor[i]}\ \text{scalefactor[j]}\  a_{ij} \\
\bar{\varepsilon}_{ij} &= \text{scalefactor[i]}\ \text{scalefactor[j]}\  h_{ij},
\end{align}

where in this case we *multiply* due to the fact that these tensors are purely covariant (as opposed to contravariant).

To slightly simplify the notation, in NRPy+ we define *rescaling variable* ReU\[i\] and ReDD\[i\]\[j\], such that

\begin{align}
\text{ReU[i]} &= 1 / \text{scalefactor[i]} \\
\text{ReDD[i][j]} &= \text{scalefactor[i] scalefactor[j]}.
\end{align}

Thus, for example,
\begin{align}
\bar{A}_{ij} &= \text{ReDD[i][j]} a_{ij} \\
\bar{\Lambda}^{i} &= \text{ReU[i]} \lambda^i,
\end{align}
where no sums are implied by the repeated indices.

Further, since the scale factors are *time independent*, 

$$\partial_t \bar{A}_{ij} = \text{ReDD[i][j]}\  \partial_t a_{ij}.$$

Thus in the below implementation of the BSSN equations, we will first define the right-hand sides of the equations for all "evolved" quantities 

$$\left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\},$$ 

and then divide or multiply the indexed evolved quantities by the scale factors according to this prescription so that the evolved variables *in our numerical scheme* are

$$\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\},$$ 
respectively.

Additionally, to ensure that *spatial* derivatives do not sample over a coordinate singularity (where there may be, e.g., a divergence), we simply apply the product/quotient rule. For example,

\begin{align}
\bar{\Lambda}^{i}_{\, ,\, j} &= -\frac{\lambda^i}{(\text{ReU[i]})^2}\ \partial_j \left(\text{ReU[i]}\right) + \frac{\partial_j \lambda^i}{\text{ReU[i]}} \\
&= -\frac{\lambda^i}{(\text{ReU[i]})^2}\ \text{ReU_dD[i][j]} + \frac{\partial_j \lambda^i}{\text{ReU[i]}}
\end{align}

where the derivative $\text{ReU_dD[i][j]}$ **is computed symbolically and exactly** using SymPy, and the derivative $\partial_j \lambda^i$ represents a derivative of a *smooth* quantity (so long as $\bar{\Lambda}^{i}$ is smooth in a Cartesian basis).

In [1]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm
from outputC import *

In [2]:
# Step 2a: Initialize BSSN_RHS parameters

# Step 2b: Set desired spatial dimension
par.set_paramsvals_value("grid::DIM = 3")

# Step 3: Register all needed *evolved* gridfunctions.
# Step 3a: Register indexed quantities, using ixp.register_... functions
hDD = ixp.register_gridfunctions_for_single_rank2("EVOL","hDD", "sym12")
aDD = ixp.register_gridfunctions_for_single_rank2("EVOL","aDD", "sym12")
lambdaU = ixp.register_gridfunctions_for_single_rank1("EVOL","lambdaU")
vetU = ixp.register_gridfunctions_for_single_rank1("EVOL","vetU")
betU = ixp.register_gridfunctions_for_single_rank1("EVOL","betU")
# Step 3b: Register scalar quantities, using gri.register_gridfunctions()
trK, cf, alpha = gri.register_gridfunctions("EVOL",["trK","cf","alpha"])

# Step 4: Register all *auxiliary* gridfunctions.
# Step 4a: Register indexed quantities, using ixp.register_... functions
RbarDD = ixp.register_gridfunctions_for_single_rank2("EVOL","RbarDD", "sym12")
# Step 4b: Register scalar quantities, using gri.register_gridfunctions()
detg = gri.register_gridfunctions("AUX",["detg"])

## Numerical Implementation: Spatial derivatives of barred quantities

As described above, barred quantities may be expressed in terms of products or quotients of reference metric scale factors multiplied by smooth quantities. Thus derivatives of barred quantities must be expanded in terms of an approximate derivative of a smooth quantity multiplied by an exact expression for the derivative of the scale factor (actually, the rescaling variables ReU\[i\] and ReDD\[i\]\[j\], which are trivially defined in terms of the scale factors, above).

### Computing the $\hat{D}_{k} \hat{D}_{l} \bar{\gamma}_{i j}$ term in $\bar{R}_{ij}$

This is perhaps the most difficult term to compute, so we work out the arithmetic below.

First, the covariant derivative of a tensor is given by (from the [wikipedia article on covariant differentiation](https://en.wikipedia.org/wiki/Covariant_derivative)):
\begin{align}
  {(\nabla_{e_c} T)^{a_1 \ldots a_r}}_{b_1 \ldots b_s} = {}
    &\frac{\partial}{\partial x^c}{T^{a_1 \ldots a_r}}_{b_1 \ldots b_s} \\
    &+ \,{\Gamma ^{a_1}}_{dc} {T^{d a_2 \ldots a_r}}_{b_1 \ldots b_s} + \cdots + {\Gamma^{a_r}}_{dc} {T^{a_1 \ldots a_{r-1}d}}_{b_1 \ldots b_s} \\
    &-\,{\Gamma^d}_{b_1 c} {T^{a_1 \ldots a_r}}_{d b_2 \ldots b_s} - \cdots - {\Gamma^d}_{b_s c} {T^{a_1 \ldots a_r}}_{b_1 \ldots b_{s-1} d}.
\end{align}

Therefore, 

$$\hat{D}_{l} \bar{\gamma}_{i j} = \bar{\gamma}_{i j,l} - \hat{\Gamma}^m_{i l} \bar{\gamma}_{m j} -\hat{\Gamma}^m_{j l} \bar{\gamma}_{i m},$$
where
\begin{align}
\bar{\gamma}_{i j} &= \hat{\gamma}_{ij} + \varepsilon_{ij} \\
&= \hat{\gamma}_{ij} + h_{ij} \text{ReDD[i][j]}.
\end{align}
So the partial derivative $\bar{\gamma}_{i j, k}$ is given by
$$\bar{\gamma}_{i j, k} = \hat{\gamma}_{ij,k} + h_{ij,k} \text{ReDD[i][j]}  + h_{ij} \text{ReDD_dD[i][j][k]}.$$

Putting it together,
$$\hat{D}_{l} \bar{\gamma}_{i j} = \hat{\gamma}_{ij,l} + h_{ij,l} \text{ReDD[i][j]}  + h_{ij} \text{ReDD_dD[i][j][l]} - \hat{\Gamma}^m_{i l} \bar{\gamma}_{m j} -\hat{\Gamma}^m_{j l} \bar{\gamma}_{i m}$$

Next we perform the second covariant derivative:



In [3]:
# Step 5: Define 